# Editing Dashboard

This notebook is intended to give an interactive dashboard to view changes to the phenotype when changing the DNA

In [15]:
import base64
from pathlib import Path
import functools

import numpy as np

from gtp.configs.loaders import load_configs
from gtp.configs.project import GenotypeToPhenotypeConfigs
from gtp.dataloading.path_collectors import (
    get_post_processed_genotype_directory,
    get_results_plot_output_directory,
    get_results_training_metadata_directory,
)
from gtp.dataloading.tools import collect_chromosome_position_metadata
from gtp.options.plot_attribution import PlotAttributionOptions

config_path = Path("../configs/default.yaml")
cfgs: GenotypeToPhenotypeConfigs = load_configs(config_path)
opts: PlotAttributionOptions = PlotAttributionOptions(
    species="erato",
    color="color_3",
    wing="forewings",
    exp_name="base",
    verbose=True,
)


# Using base64 encoding and decoding
def b64_image(image_filename):
    with open(image_filename, "rb") as f:
        image = f.read()
    return "data:image/png;base64," + base64.b64encode(image).decode("utf-8")


def get_attribution_plot():
    plot_output_dir = get_results_plot_output_directory(cfgs.io)
    plot_output_dir_final = (
        plot_output_dir / opts.exp_name / f"{opts.species}_{opts.wing}_{opts.color}"
    )
    plot_output_dir_final.mkdir(exist_ok=True, parents=True)
    img_path = plot_output_dir_final / "manhattan_plot_Attribution.png"
    if not img_path.exists():
        # Backwards compatibility from recent change. Delete this if after re run of experiments
        img_path = plot_output_dir_final / "manhattan_plot.png"

    return b64_image(img_path)


def load_camids():
    metadata_dir = get_results_training_metadata_directory(cfgs.io)
    train_camids = np.load(metadata_dir / f"{opts.species}_train.npy").tolist()
    val_camids = np.load(metadata_dir / f"{opts.species}_val.npy").tolist()
    test_camids = np.load(metadata_dir / f"{opts.species}_test.npy").tolist()
    camids = train_camids + val_camids + test_camids

    return camids


def load_genes():
    genotype_folder = get_post_processed_genotype_directory(cfgs.io)

    position_metadata = collect_chromosome_position_metadata(
        genotype_folder / cfgs.experiment.genotype_scope,
        opts.species,
        opts.chromosome,
    )

    genes = [f"{scaffold}:{position}" for scaffold, position in position_metadata]

    return genes[:50]

In [16]:
from dash import Dash, html, dcc, callback, Output, Input, jupyter_dash
import plotly.express as px
import pandas as pd

jupyter_dash.default_mode = "external"

species = ["erato", "melpomene"]
wings = ["forewings", "hindwings"]
phenotypes = ["color_1", "color_2", "color_3", "total"]
PCA_DIMS = 1
pca_dim = [i for i in range(PCA_DIMS)]
gene_edits = ["AA", "Aa/aA", "aa"]

app = Dash()

app.layout = [
    html.H1(children="Gene Editing Dashboard", style={"textAlign": "center"}),
    dcc.Dropdown(species, opts.species, id="species-selection"),
    dcc.Dropdown(wings, opts.wing, id="wing-selection"),
    dcc.Dropdown(phenotypes, opts.color, id="phenotype-selection"),
    dcc.Dropdown(pca_dim, 0, id="output-dim-selection"),
    html.H1(id="custom_text", children="Template"),
    html.Img(id="attribution-plot", src=get_attribution_plot(), width=600, height=400),
    dcc.Dropdown([], "", id="specimen-selection"),
    dcc.Dropdown([], "", id="gene-selection"),
    dcc.Dropdown(gene_edits, "", id="gene-edit-selection"),
]


def create_title_str():
    return f"{opts.species}-{opts.wing}-{opts.color}-{0}"


@callback(
    Output("custom_text", "children"),
    Input("species-selection", "value"),
    Input("wing-selection", "value"),
    Input("phenotype-selection", "value"),
    Input("output-dim-selection", "value"),
)
def update_model(species, wing, phenotype, output_dim):
    opts.species = species
    opts.wing = wing
    opts.color = phenotype
    return [create_title_str()]


@callback(
    Output("attribution-plot", "src"),
    Input("species-selection", "value"),
    Input("wing-selection", "value"),
    Input("phenotype-selection", "value"),
    Input("output-dim-selection", "value"),
)
def update_plot(species, wing, phenotype, output_dim):
    opts.species = species
    opts.wing = wing
    opts.color = phenotype
    return get_attribution_plot()


@callback(
    Output("specimen-selection", "options"),
    Output("specimen-selection", "value"),
    Input("species-selection", "value"),
    Input("wing-selection", "value"),
    Input("phenotype-selection", "value"),
    Input("output-dim-selection", "value"),
)
def update_specimen_selections(species, wing, phenotype, output_dim):
    opts.species = species
    opts.wing = wing
    opts.color = phenotype
    camids = load_camids()
    return camids, camids[0]


@callback(
    Output("gene-selection", "options"),
    Output("gene-selection", "value"),
    Input("species-selection", "value"),
    Input("wing-selection", "value"),
    Input("phenotype-selection", "value"),
    Input("output-dim-selection", "value"),
)
def update_gene_selections(species, wing, phenotype, output_dim):
    opts.species = species
    opts.wing = wing
    opts.color = phenotype
    genes = load_genes()
    return genes, genes[0]


if __name__ == "__main__":
    app.run(debug=True)

Dash app running on http://127.0.0.1:8050/
